# TE field in a lossy cavity

$\newcommand{\veps}{\varepsilon}
\newcommand{\og}{\omega}
\newcommand{\ii}{\imath}
\newcommand{\Jcl}{J_{\pi/2}}
\renewcommand{\d}{\partial}
\DeclareMathOperator{\rot}{rot}
\DeclareMathOperator{\grad}{grad}
\DeclareMathOperator{\curl}{curl}
\DeclareMathOperator{\dive}{div}
$

Recall the Maxwell system for the time-harmonic electric ($E$) and magnetic ($H$) fields,

\begin{align}
    -\ii \og \mu H + \curl E & = 0, \\
    \ii \og \veps E + \curl H & = - \sigma E - J^a.
\end{align}

which represent six equations counting scalar components.  The system is driven by  $J^a,$  the externally applied current, and we are given the frequency $\og,$  the electric permittivity $\veps$, the magnetic permeability $\mu$, and the electric conductivity $\sigma$.

In the presence of translational symmetry, the above system *decouples* into two systems, each of three equations, often known as the **TE (transverse electric)** and the **TM (transverse magnetic)** systems. In this activity, we derive the TE system, its weak formulation and finite element approximation, and numerically  solve it within an infinite lossy  tube.

## The decoupling

The three-dimensional (3D) curl can be written in terms of the 2D curl when there is  translational symmetry.   To do so, let  $e_x, e_y$ and $e_z$ denote the   coordinate unit vectors in $x, y$ and $z$ directions. 
Recall the 2D curl we defined in [a previous notebook](VectorFE.ipynb) using the operator $\Jcl$ that rotates vectors clockwise by 90 degrees: 
$$
\curl u = \dive (\Jcl u) = \d_x u_y - \d_y u_x.
$$
We will also need the *rotated 2D gradient* of a scalar-valued function $\phi$, defined by  
$$
\rot \phi = \Jcl (\grad \phi) =  \begin{bmatrix}
      \d_y \phi \\
      -\d_x \phi
    \end{bmatrix}
$$

If a smooth 3D vector field $E = E_x e_x + E_y e_y + E_z e_z$ has  translational symmetry in $z$-direction (i.e., if $\d_z E$ vanishes),   then the application of the 3D curl reduces to applications of the  2D curl and rot as follows:
$$
    \curl E =
    \begin{bmatrix}
      \rot E_z  \\ \curl E_{xy}
    \end{bmatrix}
$$
  where $E_{xy} = E_x e_x + E_y e_y$.

The above identity is all we need to witness the decoupling. Replacing every $\curl$ in the 3D Maxwell system using this identity, simple calculations show the following.


**Proposition.** <font color=blue>If the domain, $J^a, \veps, \sigma, \mu$ all have translational  symmetry in the $z$-direction, then the six-equation 3D Maxwell  system    decouples into the three-equation TM system
    \begin{align}
      -\ii \og \mu H_{xy}  + \rot E_z & = 0, \\
      \ii \og (\veps + \frac{\sigma}{\ii\og}) E_z + \curl H_{xy} & = - J^a_z,
    \end{align}
and the three-equation TE system
      \begin{align}
        -\ii \og \mu H_z  + \curl E_{xy} & = 0, \\
        \ii \og (\veps + \frac{\sigma}{\ii\og}) E_{xy} + \rot H_z & = - J^a_{xy}.
      \end{align}    
</font>

Eliminating $H_z$ from the last system, we obtain a system solely for the 
components of the electric field transverse to the direction of symmetry, namely $E_{xy}$ satisfies
$$
\rot\mu^{-1} \curl E_{xy} - (\og^2\veps - \ii \og\sigma) E_{xy} = - \ii \og J_{xy}^a.
$$

In the remainder we focus on computing $E_{xy}$ for a specific example. Henceforth we drop all the $xy$ subscripts. 

## An infinite tube

Consider an infinite tube, whose translational symmetry direction is $z$, of a circular ($xy$) cross section $\Omega$ of radius 2, enclosing two conducting layers, and enclosed by a perfect conductor. Here are further details that provide an idealized, somewhat contrived example:

- The tube material is a dielectric known to have constant $\mu$ and $\veps$, so we may multiply through the previous equation for the TE field by $\mu$. Furthermore, since the tube is enclosed by a perfect conductor, we may  use the perfect electric boundary condition on  $\d\Omega$. *(How will you express the the perfect electric boundary condition using the transverse field?)*  Accordingly, the boundary value problem for the TE field $E$ is  
\begin{align}
\rot \curl E - (\og^2\veps\mu - \ii \og\mu\sigma) E 
& = - \ii \og\mu J^a, && \text { in } \Omega,
\\
E \cdot t & = 0, && \text { on } \d\Omega,
\end{align}
where $t$ is a unit tangent to the boundary.

- We are given that the *wavenumber* $k^2 = \og^2 \veps \mu$ is given by 
$$
k = 15.
$$

- The tube has a low-loss central region and a highly absorbing outer annulus. This is modeled by setting a piecewise constant $s = \og\sigma \mu$ of the form 
$$
s =  \left\{
\begin{aligned}
& 0.1 && \text{ if } r > 1.5 \\
& 300 && \text{ otherwise}.
\end{aligned}
\right.
$$


- Suppose a time-harmonic current pulse is set centered in the tube so that 
$$
f = -\ii \og J^a = 10 \exp(-100( x^2 + y^2)) e_x.
$$

Let us compute a finite element approximation to the resulting TE field $E$.

## Weak form of the TE system

Multiplying by a test function $v$ and integrating by parts *(what happens when you integrate $\rot$ by parts?)* we obtain the following weak formulation for the TE field $E$.
$\newcommand{\Ho}{\mathring{H}}
\newcommand{\om}{\Omega}$

>Find  $E \in \Ho(\curl, \om)$ satisfying
>$$ (\curl E, \curl v) -  ((k^2 - \ii s) E, v) = (f, v)
\qquad \text{ for all } v \in \Ho(\curl, \om).$$

Here $(\cdot, \cdot)$ denotes the complex $L^2$ inner product. Since $s$ is nonzero, this is a coercive weak form, so Lax-Milgram and Cea lemmas can be applied as usual to satisfactorily complete the theory, once conforming finite elements are used. 

## Finite element solution

The above weak form directly leads to a finite element method  using the $H(\curl)$-conforming Nedelec elements, in much the same way as we have seen for other boundary values problems.
This is implemented below. 

In [ ]:
import ngsolve as ng
from ngsolve import curl, dx
from ngsolve.webgui import Draw
from netgen.webgui import Draw as DrawGeo
from netgen.occ import Circle, OCCGeometry, X, Y
from ngsolve import x, y, CF

In [ ]:
r = 2
c = Circle((0, 0), r).Face()
c.edges[0].name='out'
g = OCCGeometry(c, dim=2)
mesh = ng.Mesh(g.GenerateMesh(maxh=0.1))

pulse = 10*ng.exp(-100*( x*x + y*y))
pls = ng.CoefficientFunction((pulse, 0))

Here is a picture of the concentrated Gaussian type source due to the centered current and the underlying mesh.

In [ ]:
Draw(pulse, mesh, 'pulse');

The two conducting layers are represented as follows.

In [ ]:
r = ng.sqrt(x*x + y*y)
s = ng.IfPos(r - 1.5, 300, 0.1)
# Draw(s, mesh)

To solve the problem, we proceed as usual to assemble the finite element system of the equation $(\curl E, \curl v) -  ((k^2 - \ii s) E, v) = (f, v)$ using the NGSolve facilities we have learned earlier.

In [ ]:
k = 15

X = ng.HCurl(mesh, order=4, type1=True, complex=True, dirichlet='out')
u,v = X.TnT()

a = ng.BilinearForm(X)
a += (curl(u) * curl(v) - (k**2 - 1j*s) * u*v) * dx
f = ng.LinearForm(X)
f += pls * v * dx
    
E = ng.GridFunction(X, name='TE field')

with ng.TaskManager():
    a.Assemble()
    f.Assemble()
    
# Solve:     
E.vec.data = a.mat.Inverse(X.FreeDofs()) * f.vec

Since $E$ is a complex vector field, in order to visualize the computed solution, we usually look at its real and imaginary parts.  Here is the real part. (To see it as a quiver plot, include the `vectors` keyword argument.)

In [ ]:
Draw(E.real, mesh); #, vectors={'grid_size': 100})

Observe that the solution clearly displays a *wave* character in space. Of course, we expected wave character in time, due to the time-harmonic assumption, but the solution also shows wave character in space. If you are not so familiar with the Maxwell system, it's natural to wonder why it admits solutions that are wavy in space, even when the source we provided was a pulse with no spatial wave character. There is no mystery once you understand the intimate connection between the Maxwell system and the wave equation: do  Exercise 1 below.

Also note from the solution plot that the real part of the TE field decays after it enters the outer absorbing region of higher conductivity. In fact all components of the TE field decays in this region: we can check this by plotting the pointwise *norm* of the (complex) TE field.

In [ ]:
Draw(E, mesh, eval='norm');

Recall that the physical electric field is (not complex, but) given by the real part of the complex time-harmonic field multiplied by $\exp(-\ii\og t)$. You can get an idea of this harmonic time variation by plotting the real field `E.real` and asking NGSolve to animate it.
This can be accomplished by opening the controls box above, selecting `eval` drop-down to `real`, and then checking the `animate` button under the `Complex` tab. 

## Exercises

**Exercise 1.** Suppose a 2D vector field $u = u_x e_x + u_y e_y$ solves $\rot \curl u - k^2 u = f$ for some $f$ with $\dive f =0$ and some constant wavenumber $k$. Show that each component of the vector field $U(x, y, t) = u(x, y) e^{-\ii k t}$ solves the wave equation: $\d_{tt} U - \Delta U = f e^{-\ii k t}$.


**Exercise 2.** Consider a configuration of data and 3D domain that is rotationally symmetric about the $z$-axis and examine Maxwell solutions that satisfy $\d_\theta E = \d_\theta H = 0$, where $\theta$ is the azimuthal angle in the cylindrical coordinate system. For this case, prove an analogue of the proposition in this notebook, formulating the analogues of TE and TM systems in cylindrical coordinates.

<hr>

    
$\ll$ [Table Of Contents](./INDEX.ipynb)
<br>
$\ll$ [Jay Gopalakrishnan](http://web.pdx.edu/~gjay/)
